In [1]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 9818, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 9818 (delta 6), reused 6 (delta 0), pack-reused 9796
Receiving objects: 100% (9818/9818), 10.27 MiB | 11.33 MiB/s, done.
Resolving deltas: 100% (6782/6782), done.
/content/yolov5
     |████████████████████████████████| 596 kB 12.9 MB/s 


In [2]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 13.1 MB/s 
     |████████████████████████████████| 139 kB 40.8 MB/s 
     |████████████████████████████████| 180 kB 52.1 MB/s 
     |████████████████████████████████| 97 kB 7.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=80016921da623b935ba5aab4a8a35c7c0d3a94fef8e94c315be7707b63dd46f6
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=22a3690bd1fa98f5b1c72f1ed107b909a3ade6e8a141f6a94a2355656ff6e2d3
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import shutil

In [5]:
from torchsummary import summary
summary(model)

NameError: ignored

In [ ]:
def FindBatchSize(model):
    """model: model architecture, that is yet to be trained"""
    import os, sys, psutil, gc, tensorflow, keras
    import numpy as np
    from keras import backend as K
    BatchFound= 16

    try:
        total_params= int(model.count_params());    GCPU= "CPU"
        #find whether gpu is available
        try:
            if K.tensorflow_backend._get_available_gpus()== []:
                GCPU= "CPU";    #CPU and Cuda9GPU
            else:
                GCPU= "GPU"
        except:
            from tensorflow.python.client import device_lib;    #Cuda8GPU
            def get_available_gpus():
                local_device_protos= device_lib.list_local_devices()
                return [x.name for x in local_device_protos if x.device_type == 'GPU']
            if "gpu" not in str(get_available_gpus()).lower():
                GCPU= "CPU"
            else:
                GCPU= "GPU"

        #decide batch size on the basis of GPU availability and model complexity
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params <1000000):
            BatchFound= 64    
        if (os.cpu_count() <16) and (total_params <500000):
            BatchFound= 64  
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params <2000000) and (total_params >=1000000):
            BatchFound= 32      
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params >=2000000) and (total_params <10000000):
            BatchFound= 16  
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params >=10000000):
            BatchFound= 8       
        if (os.cpu_count() <16) and (total_params >5000000):
            BatchFound= 8    
        if total_params >100000000:
            BatchFound= 1

    except:
        pass
    try:

        #find percentage of memory used
        memoryused= psutil.virtual_memory()
        memoryused= float(str(memoryused).replace(" ", "").split("percent=")[1].split(",")[0])
        if memoryused >75.0:
            BatchFound= 8
        if memoryused >85.0:
            BatchFound= 4
        if memoryused >90.0:
            BatchFound= 2
        if total_params >100000000:
            BatchFound= 1
        print("Batch Size:  "+ str(BatchFound));    gc.collect()
    except:
        pass

    memoryused= [];    total_params= [];    GCPU= "";
    del memoryused, total_params, GCPU;    gc.collect()
    return BatchFound

In [ ]:
batch_sizes = 
[16,
 20,
 32,
 40,
 64,
 80,
 96,
 128 #RuntimeError: CUDA out of memory. Tried to allocate 58.00 MiB (GPU 0; 11.17 GiB total capacity; 10.34 GiB already allocated; 27.81 MiB free; 10.59 GiB reserved in total by PyTorch)
 ]

In [ ]:
!python train.py --img 480 --batch 96 --epochs 300 --data /content/gdrive/MyDrive/CE_Project/Dataset/labeled/dataset.yaml --weights yolov5s.pt
shutil.copytree('/content/yolov5/runs','/content/gdrive/MyDrive/CE_Project/model')

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=/content/gdrive/MyDrive/CE_Project/Dataset/labeled/dataset.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=300, batch_size=96, imgsz=480, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-42-g4c0982a torch 1.9.0+cu111 CUDA:0 (Tesla K80, 11441.1875MB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.93

#test 

In [ ]:
import torch

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5x')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 2021-9-18 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)




requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



  0%|          | 0.00/168M [00:00<?, ?B/s]

Fusing layers... 
Model Summary: 476 layers, 87730285 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
img = 'https://ultralytics.com/images/zidane.jpg' 

In [ ]:
results = model(img)

# Results
results.print()

image 1/1: 720x1280 2 persons, 2 ties
Speed: 317.2ms pre-process, 205.0ms inference, 2.7ms NMS per image at shape (1, 3, 384, 640)


In [ ]:
!python detect.py --source https://ultralytics.com/images/zidane.jpg

detect: weights=yolov5s.pt, source=https://ultralytics.com/images/zidane.jpg, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-444-g302a1b0 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
cv2.imshow() is disabled in Google Colab environments
1/1: https://ultralytics.com/images/zidane.jpg...  success (1 frames 1280x720 at 25.00 FPS)

Traceback (most recent call last):
  File "detect.py", line 293, in <module>
    main(opt)
  File "detect.py", line 288, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py", line 28, in decorate_context
    return func(*

In [ ]:
# /content/yolov5/runs